In [1]:
import pandas as pd
import numpy as np
import math
import random as python_random
import io
import os
import glob
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
from IPython.display import clear_output

seeds=[19,31,38,47,77]

In [2]:
diseases = ['No Finding']

diseases_abbr = {'No Finding': 'No Finding'}

age_decile = ['60-80', '40-60', '20-40', '80+', '0-20']

gender = ['M', 'F']

race = ['WHITE', 'BLACK/AFRICAN AMERICAN',
        'HISPANIC/LATINO', 'OTHER', 'ASIAN',
        'AMERICAN INDIAN/ALASKA NATIVE']

insurance = ['Medicare', 'Other', 'Medicaid']

factor = [gender, age_decile, race, insurance]

factor_str = ['gender', 'age_decile', 'race', 'insurance']

In [3]:
###  FPR
def fpr(df, d, c, category_name):
    pred_disease = "bi_" + d
    gt = df.loc[(df[d] == 0) & (df[category_name] == c), :]
    pred = df.loc[(df[pred_disease] == 1) & (df[d] == 0) & (df[category_name] == c), :]
    if len(gt) != 0:
        FPR = len(pred) / len(gt)
        return FPR
    else:
        # print("Disease", d, "in category", c, "has zero division error")
        return -1


In [4]:
def FPR_NF(df, diseases, category, category_name,seed=19,fpr_npr_path="default"):

  #return FPR and FNR per subgroup and the unber of patients with 0 No-finding in test set.
  FP_total = []
  percentage_total = []
  FN_total = []


  if category_name == 'race':
    FPR_race = pd.DataFrame(diseases, columns=["diseases"])

  if category_name == 'gender':
    FPR_sex = pd.DataFrame(diseases, columns=["diseases"])

  if category_name == 'age_decile':
    FPR_age = pd.DataFrame(diseases, columns=["diseases"])

  print("FP in CheXpert====================================")

  for c in category:
    FP_y = []
    FN_y = []
    percentage_y = []

    for d in diseases:
      pred_disease = "bi_" + d
      gt_fp = df.loc[(df[d] == 0) & (df[category_name] == c), :]
      gt_fn = df.loc[(df[d] == 1) & (df[category_name] == c), :]
      pred_fp = df.loc[(df[pred_disease] == 1) & (df[d] == 0) & (df[category_name] == c), :]
      pred_fn = df.loc[(df[pred_disease] == 0) & (df[d] == 1) & (df[category_name] == c), :]

      #number of patient in subgroup with actual NF=0
      pi_gy = df.loc[(df[d] == 0) & (df[category_name] == c), :]

      if len(gt_fp) != 0 :

        FPR = len(pred_fp) / len(gt_fp)
        Percentage = len(pi_gy)
        FP_y.append(round(FPR,3))
        percentage_y.append(round(Percentage,3))
        # print(c)
        print(len(pred_fp),'--' ,len(gt_fp),'====',c)
        # print("FPR in " + category[c]  + " for " + diseases[d] + " is: " + str(FPR))

      else:

        FP_y.append(np.NaN)
        percentage_y.append(0)

      if len(gt_fn) != 0 :
        FNR = len(pred_fn) / len(gt_fn)
        FN_y.append(round(FNR,3))


      else:
        FN_y.append(np.NaN)

      FP_total.append(FP_y)
      percentage_total.append(percentage_y)
      FN_total.append(FN_y)

    # print("False Positive Rate in " + category[c] + " for " + diseases[d] + " is: " + str(FPR))

  for  i in range(len(FN_total)):

    if category_name == 'gender':
      if i == 0:
        Perc_S = pd.DataFrame(percentage_total[i], columns=["#M"])
        FPR_sex = pd.concat([FPR_sex, Perc_S.reindex(FPR_sex.index)], axis=1)

        FPR_S = pd.DataFrame(FP_total[i], columns=["FPR_M"])
        FPR_sex = pd.concat([FPR_sex, FPR_S.reindex(FPR_sex.index)], axis=1)
        FNR_S = pd.DataFrame(FN_total[i], columns=["FNR_M"])
        FPR_sex = pd.concat([FPR_sex, FNR_S.reindex(FPR_sex.index)], axis=1)

      if i == 1:
        Perc_S = pd.DataFrame(percentage_total[i], columns=["#F"])
        FPR_sex = pd.concat([FPR_sex, Perc_S.reindex(FPR_sex.index)], axis=1)

        FPR_S = pd.DataFrame(FP_total[i], columns=["FPR_F"])
        FPR_sex = pd.concat([FPR_sex, FPR_S.reindex(FPR_sex.index)], axis=1)

        FNR_S = pd.DataFrame(FN_total[i], columns=["FNR_F"])
        FPR_sex = pd.concat([FPR_sex, FNR_S.reindex(FPR_sex.index)], axis=1)
        FPR_sex.to_csv(fpr_npr_path+"run_"+str(seed)+"FPR_FNR_NF_sex.csv")

    if category_name == 'age_decile':

      if i == 0:
        Perc_A = pd.DataFrame(percentage_total[i], columns=["#60-80"])
        FPR_age = pd.concat([FPR_age, Perc_A.reindex(FPR_age.index)], axis=1)

        FPR_A = pd.DataFrame(FP_total[i], columns=["FPR_60-80"])
        FPR_age = pd.concat([FPR_age, FPR_A.reindex(FPR_age.index)], axis=1)

        FNR_A = pd.DataFrame(FN_total[i], columns=["FNR_60-80"])
        FPR_age = pd.concat([FPR_age, FNR_A.reindex(FPR_age.index)], axis=1)

      if i == 1:

        Perc_A = pd.DataFrame(percentage_total[i], columns=["#40-60"])
        FPR_age = pd.concat([FPR_age, Perc_A.reindex(FPR_age.index)], axis=1)

        FPR_A = pd.DataFrame(FP_total[i], columns=["FPR_40-60"])
        FPR_age = pd.concat([FPR_age, FPR_A.reindex(FPR_age.index)], axis=1)

        FNR_A = pd.DataFrame(FN_total[i], columns=["FNR_40-60"])
        FPR_age = pd.concat([FPR_age, FNR_A.reindex(FPR_age.index)], axis=1)

      if i == 2:

        Perc_A = pd.DataFrame(percentage_total[i], columns=["#20-40"])
        FPR_age = pd.concat([FPR_age, Perc_A.reindex(FPR_age.index)], axis=1)

        FPR_A = pd.DataFrame(FP_total[i], columns=["FPR_20-40"])
        FPR_age = pd.concat([FPR_age, FPR_A.reindex(FPR_age.index)], axis=1)
        FNR_A = pd.DataFrame(FN_total[i], columns=["FNR_20-40"])
        FPR_age = pd.concat([FPR_age, FNR_A.reindex(FPR_age.index)], axis=1)

      if i == 3:
        Perc_A = pd.DataFrame(percentage_total[i], columns=["#80+"])
        FPR_age = pd.concat([FPR_age, Perc_A.reindex(FPR_age.index)], axis=1)
        FPR_A = pd.DataFrame(FP_total[i], columns=["FPR_80+"])
        FPR_age = pd.concat([FPR_age, FPR_A.reindex(FPR_age.index)], axis=1)
        FNR_A = pd.DataFrame(FN_total[i], columns=["FNR_80+"])
        FPR_age = pd.concat([FPR_age, FNR_A.reindex(FPR_age.index)], axis=1)

      if i == 4:

        Perc_A = pd.DataFrame(percentage_total[i], columns=["#0-20"])
        FPR_age = pd.concat([FPR_age, Perc_A.reindex(FPR_age.index)], axis=1)

        FPR_A = pd.DataFrame(FP_total[i], columns=["FPR_0-20"])
        FPR_age = pd.concat([FPR_age, FPR_A.reindex(FPR_age.index)], axis=1)

        FNR_A = pd.DataFrame(FN_total[i], columns=["FNR_0-20"])
        FPR_age = pd.concat([FPR_age, FNR_A.reindex(FPR_age.index)], axis=1)
        FPR_age.to_csv(fpr_npr_path+"run_"+str(seed)+"FPR_FNR_NF_age.csv")

    if category_name == 'race':

      if i == 0:

        Perc_A = pd.DataFrame(percentage_total[i], columns=["#White"])
        FPR_race = pd.concat([FPR_race, Perc_A.reindex(FPR_race.index)], axis=1)

        FPR_A = pd.DataFrame(FP_total[i], columns=["FPR_White"])
        FPR_race = pd.concat([FPR_race, FPR_A.reindex(FPR_race.index)], axis=1)

        FNR_A = pd.DataFrame(FN_total[i], columns=["FNR_White"])
        FPR_race = pd.concat([FPR_race, FNR_A.reindex(FPR_race.index)], axis=1)


      if i == 1:
        Perc_A = pd.DataFrame(percentage_total[i], columns=["#Black"])
        FPR_race = pd.concat([FPR_race, Perc_A.reindex(FPR_race.index)], axis=1)

        FPR_A = pd.DataFrame(FP_total[i], columns=["FPR_Black"])
        FPR_race = pd.concat([FPR_race, FPR_A.reindex(FPR_race.index)], axis=1)

        FNR_A = pd.DataFrame(FN_total[i], columns=["FNR_Black"])
        FPR_race = pd.concat([FPR_race, FNR_A.reindex(FPR_race.index)], axis=1)


      if i == 2:
        Perc_A = pd.DataFrame(percentage_total[i], columns=["#Hisp"])
        FPR_race = pd.concat([FPR_race, Perc_A.reindex(FPR_race.index)], axis=1)

        FPR_A = pd.DataFrame(FP_total[i], columns=["FPR_Hisp"])
        FPR_race = pd.concat([FPR_race, FPR_A.reindex(FPR_race.index)], axis=1)

        FNR_A = pd.DataFrame(FN_total[i], columns=["FNR_Hisp"])
        FPR_race = pd.concat([FPR_race, FNR_A.reindex(FPR_race.index)], axis=1)

      if i == 3:
        Perc_A = pd.DataFrame(percentage_total[i], columns=["#Other"])
        FPR_race = pd.concat([FPR_race, Perc_A.reindex(FPR_race.index)], axis=1)

        FPR_A = pd.DataFrame(FP_total[i], columns=["FPR_Other"])
        FPR_race = pd.concat([FPR_race, FPR_A.reindex(FPR_race.index)], axis=1)

        FNR_A = pd.DataFrame(FN_total[i], columns=["FNR_Other"])
        FPR_race = pd.concat([FPR_race, FNR_A.reindex(FPR_race.index)], axis=1)


      if i == 4:
        Perc_A = pd.DataFrame(percentage_total[i], columns=["#Asian"])
        FPR_race = pd.concat([FPR_race, Perc_A.reindex(FPR_race.index)], axis=1)

        FPR_A = pd.DataFrame(FP_total[i], columns=["FPR_Asian"])
        FPR_race = pd.concat([FPR_race, FPR_A.reindex(FPR_race.index)], axis=1)

        FNR_A = pd.DataFrame(FN_total[i], columns=["FNR_Asian"])
        FPR_race = pd.concat([FPR_race, FNR_A.reindex(FPR_race.index)], axis=1)

      if i == 5:
        Perc_A = pd.DataFrame(percentage_total[i], columns=["#American"])
        FPR_race = pd.concat([FPR_race, Perc_A.reindex(FPR_race.index)], axis=1)

        FPR_A = pd.DataFrame(FP_total[i], columns=["FPR_American"])
        FPR_race = pd.concat([FPR_race, FPR_A.reindex(FPR_race.index)], axis=1)

        FNR_A = pd.DataFrame(FN_total[i], columns=["FNR_American"])
        FPR_race = pd.concat([FPR_race, FNR_A.reindex(FPR_race.index)], axis=1)
        FPR_race.to_csv(fpr_npr_path+"run_"+str(seed)+"FPR_FNR_NF_race.csv")


In [5]:
def FP_NF_ALL_Inter(df, diseases, category1, category_name1,category2,category_name2,seed=19,fpr_npr_path_2_group_intersection="default"):

  if (category_name1 == 'gender')  &  (category_name2 == 'race'):
    FP_RaceSex = pd.DataFrame(category2, columns=["race"])

  if (category_name1 == 'gender')  &  (category_name2 == 'age_decile'):
    FP_AgeSex = pd.DataFrame(category2, columns=["Age"])


  if (category_name1 == 'race')  &  (category_name2 == 'age_decile'):
    FP_RaceAge = pd.DataFrame(category2, columns=["age"])

  print("==================================== Calculating FP in vector embedded CXP====================================")

  i = 0

  for c1 in range(len(category1)):
    FPR_list = []
    FNR_list = []

    for c2 in range(len(category2)):

      for d in range(len(diseases)):

        pred_disease = "bi_" + diseases[d]
        gt_fp =   df.loc[((df[diseases[d]] == 0)  & (df[category_name1] == category1[c1]) &
         (df[category_name2] == category2[c2])), :]

        gt_fn =   df.loc[((df[diseases[d]] == 1)  & (df[category_name1] == category1[c1]) &
         (df[category_name2] == category2[c2])), :]

        pred_fp = df.loc[((df[pred_disease] == 1) & (df[diseases[d]] == 0) & (df[category_name1] == category1[c1]) &
                          (df[category_name2] == category2[c2])), :]
        pred_fn = df.loc[((df[pred_disease] == 0) & (df[diseases[d]] == 1) & (df[category_name1] == category1[c1]) &
                          (df[category_name2] == category2[c2])), :]

        if len(gt_fp) != 0:
          FPR = len(pred_fp) / len(gt_fp)
          print(len(pred_fp),'--' ,len(gt_fp))
          print("False Positive Rate in " + category1[c1] +"/" + category2[c2] + " for " + diseases[d] + " is: " + str(FPR))

        else:
          FPR = np.NaN
          print("False Positive Rate in " + category1[c1] +"/" + category2[c2] + " for " + diseases[d] + " is: N\A")

        print('=======================================================================================================')

      FPR_list.append(round(FPR,3))


    if (category_name1 == 'gender')  &  (category_name2 == 'age_decile'):

      if i == 0:
        FPR_SA = pd.DataFrame(FPR_list, columns=["FPR_M"])
        FP_AgeSex = pd.concat([FP_AgeSex, FPR_SA.reindex(FP_AgeSex.index)], axis=1)

      if i == 1:
        FPR_SA = pd.DataFrame(FPR_list, columns=["FPR_F"])
        FP_AgeSex = pd.concat([FP_AgeSex, FPR_SA.reindex(FP_AgeSex.index)], axis=1)

    if (category_name1 == 'gender')  &  (category_name2 == 'race'):

      if i == 0:
        FPR_SR = pd.DataFrame(FPR_list, columns=["FPR_M"])
        FP_RaceSex = pd.concat([FP_RaceSex, FPR_SR.reindex(FP_RaceSex.index)], axis=1)

      if i == 1:
        FPR_SR = pd.DataFrame(FPR_list, columns=["FPR_F"])
        FP_RaceSex = pd.concat([FP_RaceSex, FPR_SR.reindex(FP_RaceSex.index)], axis=1)

    if (category_name1 == 'race')  &  (category_name2 == 'age_decile'):

      if i == 0:

        FPR_RIn = pd.DataFrame(FPR_list, columns=["FPR_White"])
        FP_RaceAge = pd.concat([FP_RaceAge, FPR_RIn.reindex(FP_RaceAge.index)], axis=1)

      if i == 1:
        FPR_RIn = pd.DataFrame(FPR_list, columns=["FPR_Black"])
        FP_RaceAge = pd.concat([FP_RaceAge, FPR_RIn.reindex(FP_RaceAge.index)], axis=1)

      if i == 2:

        FPR_RIn = pd.DataFrame(FPR_list, columns=["FPR_Hisp"])
        FP_RaceAge = pd.concat([FP_RaceAge, FPR_RIn.reindex(FP_RaceAge.index)], axis=1)

      if i == 3:
        FPR_RIn = pd.DataFrame(FPR_list, columns=["FPR_Other"])
        FP_RaceAge = pd.concat([FP_RaceAge, FPR_RIn.reindex(FP_RaceAge.index)], axis=1)

      if i == 4:
        FPR_RIn = pd.DataFrame(FPR_list, columns=["FPR_Asian"])
        FP_RaceAge = pd.concat([FP_RaceAge, FPR_RIn.reindex(FP_RaceAge.index)], axis=1)

      if i == 5:

        FPR_RIn = pd.DataFrame(FPR_list, columns=["FPR_American"])
        FP_RaceAge = pd.concat([FP_RaceAge, FPR_RIn.reindex(FP_RaceAge.index)], axis=1)

    i += 1


  if (category_name1 == 'gender')  &  (category_name2 == 'race'):
    FP_RaceSex.to_csv(fpr_npr_path_2_group_intersection+"run_"+str(seed)+"FP_RaceSex.csv")

  if (category_name1 == 'race')  &  (category_name2 == 'age_decile'):
    FP_RaceAge.to_csv(fpr_npr_path_2_group_intersection+"run_"+str(seed)+"FP_RaceAge.csv")

  if (category_name1 == 'gender')  &  (category_name2 == 'age_decile'):
    FP_AgeSex.to_csv(fpr_npr_path_2_group_intersection+"run_"+str(seed)+"FP_AgeSex.csv")

In [6]:
def FP_NF_ALL_threegroup_Inter(df, diseases, category1, category_name1,category2,
                      category_name2,category3,category_name3,seed=19,fpr_npr_path_3_group_intersection="default"):


  if (category_name1 == 'gender')  &  (category_name2 == 'age_decile') & (category_name3 == 'race'):
    FP_RaceAgeSex = pd.DataFrame(category3, columns=["race"])

  print("==================================== Calculating FP in vector embedded mimic cxr====================================")

  i = 0

  for c1 in range(len(category1)):

    # FPR_list = []

    for c2 in range(len(category2)):
      FPR_list = []


      for c3 in range(len(category3)):

        for d in range(len(diseases)):

          pred_disease = "bi_" + diseases[d]
          gt_fp =   df.loc[((df[diseases[d]] == 0)  & (df[category_name1] == category1[c1]) &
           (df[category_name2] == category2[c2]) & (df[category_name3] == category3[c3])), :]

          gt_fn =   df.loc[((df[diseases[d]] == 1)  & (df[category_name1] == category1[c1]) &
            (df[category_name2] == category2[c2]) & (df[category_name3] == category3[c3])), :]

          pred_fp = df.loc[((df[pred_disease] == 1) & (df[diseases[d]] == 0) & (df[category_name1] == category1[c1]) &
             (df[category_name2] == category2[c2]) & (df[category_name3] == category3[c3])), :]

          pred_fn = df.loc[((df[pred_disease] == 0) & (df[diseases[d]] == 1) & (df[category_name1] == category1[c1]) &
                          (df[category_name2] == category2[c2]) & (df[category_name3] == category3[c3])), :]

          if len(gt_fp) != 0:
            FPR = len(pred_fp) / len(gt_fp)
            print(len(pred_fp),'--' ,len(gt_fp))
            print("False Positive Rate in " + category3[c3] +"/" + category1[c1] + " and "+category2[c2]+ " for " + diseases[d] + " is: " + str(FPR))

          else:
            FPR = np.NaN
            # print("False Positive Rate in " + category1[c1] +"/" + category2[c2] + " and "+category3[c3]+" for " + diseases[d] + " is: N\A")

            print('=======================================================================================================')

        FPR_list.append(round(FPR,3))
        print(FPR_list)
        # break

      if (category_name1 == 'gender')  &  (category_name2 == 'age_decile') & (category_name3 == 'race'):

        if i == 0:
          fpr_60_male = pd.DataFrame(FPR_list, columns=["FPR_60_80_M"])
          FP_RaceAgeSex = pd.concat([FP_RaceAgeSex, fpr_60_male.reindex(FP_RaceAgeSex.index)], axis=1)

        if i == 1:
          fpr_40_male = pd.DataFrame(FPR_list, columns=["FPR_40_60_M"])
          FP_RaceAgeSex = pd.concat([FP_RaceAgeSex, fpr_40_male.reindex(FP_RaceAgeSex.index)], axis=1)

        if i ==2:
          fpr_20_male = pd.DataFrame(FPR_list, columns=["FPR_20_40_M"])
          FP_RaceAgeSex = pd.concat([FP_RaceAgeSex, fpr_20_male.reindex(FP_RaceAgeSex.index)], axis=1)

        if i ==3:
          fpr_80_male = pd.DataFrame(FPR_list, columns=["FPR_80+_M"])
          FP_RaceAgeSex = pd.concat([FP_RaceAgeSex, fpr_80_male.reindex(FP_RaceAgeSex.index)], axis=1)

        if i ==4:
          fpr_0_male = pd.DataFrame(FPR_list, columns=["FPR_0_20_M"])
          FP_RaceAgeSex = pd.concat([FP_RaceAgeSex, fpr_0_male.reindex(FP_RaceAgeSex.index)], axis=1)

        if i == 5:
          fpr_60_female = pd.DataFrame(FPR_list, columns=["FPR_60_80_F"])
          FP_RaceAgeSex = pd.concat([FP_RaceAgeSex, fpr_60_female.reindex(FP_RaceAgeSex.index)], axis=1)

        if i == 6:
          fpr_40_female = pd.DataFrame(FPR_list, columns=["FPR_40_60_F"])
          FP_RaceAgeSex = pd.concat([FP_RaceAgeSex, fpr_40_female.reindex(FP_RaceAgeSex.index)], axis=1)

        if i ==7:
          fpr_20_female = pd.DataFrame(FPR_list, columns=["FPR_20_40_F"])
          FP_RaceAgeSex = pd.concat([FP_RaceAgeSex, fpr_20_female.reindex(FP_RaceAgeSex.index)], axis=1)

        if i ==8:
          fpr_80_female = pd.DataFrame(FPR_list, columns=["FPR_80+_F"])
          FP_RaceAgeSex = pd.concat([FP_RaceAgeSex, fpr_80_female.reindex(FP_RaceAgeSex.index)], axis=1)

        if i ==9:
          fpr_0_female = pd.DataFrame(FPR_list, columns=["FPR_0_20_F"])
          FP_RaceAgeSex = pd.concat([FP_RaceAgeSex, fpr_0_female.reindex(FP_RaceAgeSex.index)], axis=1)

      i+=1


  if (category_name1 == 'gender')  &  (category_name2 == 'age_decile') & (category_name3 == 'race'):
     FP_RaceAgeSex.to_csv(fpr_npr_path_3_group_intersection+"run_"+str(seed)+"FP_RaceAgeSex.csv")


In [8]:

for seed in seeds:
    
    np.random.seed(seed)
    python_random.seed(seed)
    
    prediction_results_path = "./Prediction_results/"
    fpr_npr_path = "./FPR/SubGroup_FPR/"
    os.makedirs(os.path.dirname(fpr_npr_path), exist_ok=True)
    df = pd.read_csv(f"{prediction_results_path}bipred_{seed}.csv")
    
    '''' FPR Calculation '''
    
    for i in range(len(factor)):
        FPR_NF(df,diseases,factor[i],factor_str[i],seed,fpr_npr_path)

    '''' Two group FPR '''
    fpr_npr_path_2_group_intersection = "./FPR/Two_Group_Intersection_FPR/"
    os.makedirs(os.path.dirname(fpr_npr_path_2_group_intersection), exist_ok=True)
    
    FP_NF_ALL_Inter(df, diseases, gender, 'gender',race,'race',seed,fpr_npr_path_2_group_intersection)
    FP_NF_ALL_Inter(df, diseases, gender, 'gender', age_decile, 'age_decile',seed,fpr_npr_path_2_group_intersection)
    FP_NF_ALL_Inter(df, diseases, race, 'race', age_decile, 'age_decile',seed,fpr_npr_path_2_group_intersection)

    '''' Three group FPR '''
    fpr_npr_path_3_group_intersection = "./FPR/Three_Group_Intersection_FPR/"
    os.makedirs(os.path.dirname(fpr_npr_path_3_group_intersection), exist_ok=True)
    
    FP_NF_ALL_threegroup_Inter(df, diseases,gender,'gender',age_decile, 'age_decile',race,'race',
                                    seed,fpr_npr_path_3_group_intersection)


    print(f'Seed : {seed}')


C:\Users\gebre\AppData\Local\Temp\ipykernel_3852\3194881223.py:9: DtypeWarning: Columns (1,3,6,7,23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"{prediction_results_path}bipred_{seed}.csv")


FP in CheXpert====================================
1935 -- 18025 ==== M
1722 -- 13001 ==== F
FP in CheXpert====================================
301 -- 6526 ==== 60-80
282 -- 4770 ==== 40-60
133 -- 1797 ==== 20-40
165 -- 3033 ==== 80+
6 -- 125 ==== 0-20
FP in CheXpert====================================
2326 -- 20900 ==== WHITE
602 -- 3225 ==== BLACK/AFRICAN AMERICAN
216 -- 1256 ==== HISPANIC/LATINO
276 -- 2941 ==== OTHER
202 -- 2361 ==== ASIAN
35 -- 343 ==== AMERICAN INDIAN/ALASKA NATIVE
FP in CheXpert====================================
1282 -- 7287 ==== Medicare
1237 -- 6356 ==== Other
251 -- 1132 ==== Medicaid
==================================== Calculating FP in vector embedded CXP====================================
1295 -- 12355
False Positive Rate in M/WHITE for No Finding is: 0.1048158640226629
265 -- 1635
False Positive Rate in M/BLACK/AFRICAN AMERICAN for No Finding is: 0.1620795107033639
116 -- 737
False Positive Rate in M/HISPANIC/LATINO for No Finding is: 0.15739484396200

C:\Users\gebre\AppData\Local\Temp\ipykernel_3852\3194881223.py:9: DtypeWarning: Columns (1,3,6,7,23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"{prediction_results_path}bipred_{seed}.csv")


FP in CheXpert====================================
1935 -- 18025 ==== M
1722 -- 13001 ==== F
FP in CheXpert====================================
301 -- 6526 ==== 60-80
282 -- 4770 ==== 40-60
133 -- 1797 ==== 20-40
165 -- 3033 ==== 80+
6 -- 125 ==== 0-20
FP in CheXpert====================================
2326 -- 20900 ==== WHITE
602 -- 3225 ==== BLACK/AFRICAN AMERICAN
216 -- 1256 ==== HISPANIC/LATINO
276 -- 2941 ==== OTHER
202 -- 2361 ==== ASIAN
35 -- 343 ==== AMERICAN INDIAN/ALASKA NATIVE
FP in CheXpert====================================
1282 -- 7287 ==== Medicare
1237 -- 6356 ==== Other
251 -- 1132 ==== Medicaid
==================================== Calculating FP in vector embedded CXP====================================
1295 -- 12355
False Positive Rate in M/WHITE for No Finding is: 0.1048158640226629
265 -- 1635
False Positive Rate in M/BLACK/AFRICAN AMERICAN for No Finding is: 0.1620795107033639
116 -- 737
False Positive Rate in M/HISPANIC/LATINO for No Finding is: 0.15739484396200

C:\Users\gebre\AppData\Local\Temp\ipykernel_3852\3194881223.py:9: DtypeWarning: Columns (1,3,6,7,23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"{prediction_results_path}bipred_{seed}.csv")


FP in CheXpert====================================
1935 -- 18025 ==== M
1722 -- 13001 ==== F
FP in CheXpert====================================
301 -- 6526 ==== 60-80
282 -- 4770 ==== 40-60
133 -- 1797 ==== 20-40
165 -- 3033 ==== 80+
6 -- 125 ==== 0-20
FP in CheXpert====================================
2326 -- 20900 ==== WHITE
602 -- 3225 ==== BLACK/AFRICAN AMERICAN
216 -- 1256 ==== HISPANIC/LATINO
276 -- 2941 ==== OTHER
202 -- 2361 ==== ASIAN
35 -- 343 ==== AMERICAN INDIAN/ALASKA NATIVE
FP in CheXpert====================================
1282 -- 7287 ==== Medicare
1237 -- 6356 ==== Other
251 -- 1132 ==== Medicaid
==================================== Calculating FP in vector embedded CXP====================================
1295 -- 12355
False Positive Rate in M/WHITE for No Finding is: 0.1048158640226629
265 -- 1635
False Positive Rate in M/BLACK/AFRICAN AMERICAN for No Finding is: 0.1620795107033639
116 -- 737
False Positive Rate in M/HISPANIC/LATINO for No Finding is: 0.15739484396200

C:\Users\gebre\AppData\Local\Temp\ipykernel_3852\3194881223.py:9: DtypeWarning: Columns (1,3,6,7,23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"{prediction_results_path}bipred_{seed}.csv")


FP in CheXpert====================================
1935 -- 18025 ==== M
1722 -- 13001 ==== F
FP in CheXpert====================================
301 -- 6526 ==== 60-80
282 -- 4770 ==== 40-60
133 -- 1797 ==== 20-40
165 -- 3033 ==== 80+
6 -- 125 ==== 0-20
FP in CheXpert====================================
2326 -- 20900 ==== WHITE
602 -- 3225 ==== BLACK/AFRICAN AMERICAN
216 -- 1256 ==== HISPANIC/LATINO
276 -- 2941 ==== OTHER
202 -- 2361 ==== ASIAN
35 -- 343 ==== AMERICAN INDIAN/ALASKA NATIVE
FP in CheXpert====================================
1282 -- 7287 ==== Medicare
1237 -- 6356 ==== Other
251 -- 1132 ==== Medicaid
==================================== Calculating FP in vector embedded CXP====================================
1295 -- 12355
False Positive Rate in M/WHITE for No Finding is: 0.1048158640226629
265 -- 1635
False Positive Rate in M/BLACK/AFRICAN AMERICAN for No Finding is: 0.1620795107033639
116 -- 737
False Positive Rate in M/HISPANIC/LATINO for No Finding is: 0.15739484396200

C:\Users\gebre\AppData\Local\Temp\ipykernel_3852\3194881223.py:9: DtypeWarning: Columns (1,3,6,7,23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"{prediction_results_path}bipred_{seed}.csv")


FP in CheXpert====================================
1935 -- 18025 ==== M
1722 -- 13001 ==== F
FP in CheXpert====================================
301 -- 6526 ==== 60-80
282 -- 4770 ==== 40-60
133 -- 1797 ==== 20-40
165 -- 3033 ==== 80+
6 -- 125 ==== 0-20
FP in CheXpert====================================
2326 -- 20900 ==== WHITE
602 -- 3225 ==== BLACK/AFRICAN AMERICAN
216 -- 1256 ==== HISPANIC/LATINO
276 -- 2941 ==== OTHER
202 -- 2361 ==== ASIAN
35 -- 343 ==== AMERICAN INDIAN/ALASKA NATIVE
FP in CheXpert====================================
1282 -- 7287 ==== Medicare
1237 -- 6356 ==== Other
251 -- 1132 ==== Medicaid
==================================== Calculating FP in vector embedded CXP====================================
1295 -- 12355
False Positive Rate in M/WHITE for No Finding is: 0.1048158640226629
265 -- 1635
False Positive Rate in M/BLACK/AFRICAN AMERICAN for No Finding is: 0.1620795107033639
116 -- 737
False Positive Rate in M/HISPANIC/LATINO for No Finding is: 0.15739484396200